In [31]:
import c3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display
import os
PATH_TO_DATA = 'Data'
PATH_TO_RECORDED_DATA = os.path.join(PATH_TO_DATA, 'human arm tracks')
print(f'path to recorded data = {PATH_TO_RECORDED_DATA}')
# Get the csv records only
records = [f for f in sorted(os.listdir(PATH_TO_RECORDED_DATA)) if f.endswith('.csv')]
print(f'recorded list = {records}')

def read_csv_rigid_data(path):
    data = pd.read_csv(path)
    # The data records for the pig in hole tool to estimate its location
    PIH_data = data.iloc[:, :9] # rotation around (X, Y, Z) + Postition (X, Y, Z)
    return PIH_data

data = read_csv_rigid_data(os.path.join(PATH_TO_RECORDED_DATA,records[-1]))
for col in data:
    data[col] = pd.to_numeric(data[col], errors='coerce')
print(data.columns)
# for col in data:
#     data[col].fillna(value ='0', inplace=True)
data.interpolate(method ='linear', axis=1)

    
# data = data.dropna(axis=1)
display(data)

path to recorded data = Data/human arm tracks
recorded list = ['Take 2022-12-14 05.13.01 PM_001.csv', 'Take 2023-01-18 07.26.05 PM_001.csv', 'Take 2023-01-18 3.csv', 'Take 2023-01-18 4.csv', 'Take 2023-01-18 5.csv', 'Take 2023-01-18 6.csv', 'Take 2023-01-18 7.csv', 'Take 2023-01-18 8.csv', 'Take 2023-01-19 1.csv', 'Take 2023-01-19 2 processed.csv', 'Take 2023-01-19 3 processed.csv', 'Take 2023-01-19 4 processed .csv']
Index(['Frame', 'Time (Seconds)', 'PIH_X_Rotation', 'PIH_Y_Rotation',
       'PIH_Z_Rotation', 'PIH_W_Rotation', 'PIH_X_Position', 'PIH_Y_Position',
       'PIH_Z_Position'],
      dtype='object')


,Frame,Time (Seconds),PIH_X_Rotation,PIH_Y_Rotation,PIH_Z_Rotation,PIH_W_Rotation,PIH_X_Position,PIH_Y_Position,PIH_Z_Position
0,0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7504,7504,75.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7505,7505,75.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7506,7506,75.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7507,7507,75.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
del data